. | .
-- | --
![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg) | ![NASA](https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png)


<center><h1><font size="+3">ASTG Python Courses</font></h1></center>

---

<CENTER>
<H1 style="color:red">
An Introduction to netCDF4 Python
</H1>
</CENTER>

# <font color='red'> Useful References </font>

* <A HREF="http://pyhogs.github.io/intro_netcdf4.html">Create and read netCDF files</A>
* <A HREF="https://unidata.github.io/netcdf4-python/netCDF4/index.html">netCDF4 module</A>
* <a href="https://annefou.github.io/metos_python/07-LargeFiles/">Handling very large files in Python</a>

## <font color="red"> Scientific Data</font>

* N‐dimensional arrays and metadata:
* Measurements at specific time, location,condition
       – Physics: temperature, pressure
       – Chemistry: reaction speed
       – Biology: type (species, cell types, nucleotides)
       – Economics: price
       – Algorithmics: program time and space
       – Networking: network activity
       – Robotics: movements
     
**Requirements**

+ Compact storage: compression
+ Fast I/O: parallel, partial, random access
+ Portability: transporting data between computers
+ Tools for manipulating data: reorganizing, aggregating, subsetting, converting,visualizing
+ Easy API in many languages: C, C++, Fortran, Java, Matlab, Perl, Python, R, ...

## <font color="red"> What We Will Cover </font>
* Opening a file
* Dimension
* Variables
* Attributes
* Writing data
* Creating groups
* Reading data

## <font color='red'> What is netCDF?</font>

**Overview**

* The Network Common Data Form, or netCDF, is an interface to a library of data access functions for storing and retrieving data in the form of arrays.
* NetCDF is an abstraction that supports a view of data as a collection of self-describing, portable objects that can be accessed through a simple interface.
* All operations to access and manipulate data in a netCDF dataset must use only the set of functions provided by the interface.
* Array values may be accessed directly, without knowing details of how the data are stored.
* NetCDF supports efficient access to small subsets of large datasets.

**Portability**

* The netCDF library is supported for various Linux/UNIX operating systems as well as MS Windows.
* APIs written for Fortran 77/90, C, C++, Java

**Conventions**

* The mere use of netCDF is not sufficient to make data "self-describing" and meaningful to both humans and machines. 
* By using a set of conventions, a data producer is more likely to produce files that can be easily shared within the research community, and that contain enough details to be useful as a long-term archive. 
* The names of variables and dimensions should be meaningful and conform to any relevant conventions.
* It is important to use all the relevant standard attributes using the relevant conventions.



## <font color='red'> What is netCDF4 Python?</font>

* Python interface to the netCDF version 4 library.
* **Can read and write files in both the new netCDF 4 and the netCDF 3 formats**.
* Can create files that are readable by HDF5 utilities.
* Relies on NumPy arrays.

-----

**<font color="blue">Uncomment the cell below if you are in Google Colab</font>**

In [ ]:
#!pip install netCDF4

-----

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import datetime
import numpy as np
import netCDF4 as nc4

## <font color='red'> Creating a netCDF-4 File</font>

#### <font color='blue'> Opening a netCDF File</font>

In [ ]:
ncFileName = 'sample_netcdf.nc4'
modeType   = 'w'
fileFormat = 'NETCDF4'
ncfid = nc4.Dataset(ncFileName, mode=modeType, format=fileFormat)

`modeType` has the options:
* 'w': to create a new file
* 'r+': to read and write with an existing file
* 'r': to read (only) an existing file
* 'a': to append to existing file

`fileFormat` has the options: 
* 'NETCDF3_CLASSIC': Original netCDF format     
* 'NETCDF3_64BIT_OFFSET': Used to ease the size restrictions of netCDF classic files
* 'NETCDF4_CLASSIC'
* 'NETCDF4': Offer new features such as groups, compound types, variable length arrays, new unsigned integer types, parallel I/O access, etc.
* 'NETCDF3_64BIT_DATA'

#### <font color='blue'> Creating Dimensions in a netCDF File</font>
* Use the method `createDimension`
* It typically takes as arguments a string (name of the dimension) and an integer (dimension size)
* For unlimited dimensions, use `None` or `0` as size.
* Unlimited size dimensions must be declared before (“to the left of”) other dimensions.

In [ ]:
time = ncfid.createDimension('time', None)
lev  = ncfid.createDimension('lev', 72)
lat  = ncfid.createDimension('lat', 91)
lon  = ncfid.createDimension('lon', 144)

#### <font color='blue'>Create Variables</font>
* netCDF variables behave much like python multidimensional array objects supplied by the `numpy` module. However, unlike numpy arrays, netCDF4 variables can be appended to along one or more 'unlimited' dimensions. 
* To create a netCDF variable, use the method `createVariable`.
* The first three arguments are the variable name, the variable type, a tuple of previously defined dimension names.
* You can specify the datatype as a numpy dtype object, or anything that can be converted to a `numpy` dtype object. 
* If you need data compression, add the `zlib=True` argument.

**Dimension Variables**

In [ ]:
times      = ncfid.createVariable('time','f8',('time',))
levels     = ncfid.createVariable('lev','i4',('lev',))
latitudes  = ncfid.createVariable('lat','f4',('lat',))
longitudes = ncfid.createVariable('lon','f4',('lon',))

**Regular Variables**

In [ ]:
temp = ncfid.createVariable('temp','f4', 
                            ('time','lev','lat','lon',),
                            fill_value=1.0e15)

- By default, each variable is pre-filled at creation, i.e., the `_ FillValue` attribute (value that a variable gets filled with before any data is loaded into it) will automatically be set. The value depends on the type of the variable.
- If you want to set your own value of `_FillValue`, the `fill_value` argument needs to be set provided (with the appropriate value) during the variable creation. Doing so overrides the default netCDF `_FillValue`.
- If `fill_value` is et to `False`, then then the variable is not pre-filled.

#### <font color='blue'>Adding Variable Attributes</font>
* Attributes allow us to capture metadata that would otherwise be separated from the data.
* Variable attributes are added on individual variables in the dataset.
* All variables should have values for the following attributes unless there is a very good reason not to:
     - units
     - long_name

You may also consider attributes such as:
* valid_min: Smallest valid value of a variable.
* valid_max: Largest valid value of a variable.
* standard_name: A name used to identify the physical quantity. A standard name contains no whitespace and is case sensitive. 

In [ ]:
latitudes.long_name  = 'latitude'
latitudes.units      = 'degrees north'

longitudes.long_name = 'longitude'
longitudes.units     = 'degrees east'

levels.long_name     = 'vertical levels'
levels.units         = 'hPa'
levels.positive      = 'down'

beg_date = datetime.datetime(year=2019, month=1, day=1)
times.long_name      = 'time'
times.units          = beg_date.strftime('hours since %Y-%m-%d %H:%M:%S')
times.calendar       = 'gregorian'

temp.long_name       = 'temperature'
temp.units           = 'K'
temp.valid_min       = 200.
temp.valid_max       = 350.
temp.standard_name   = 'atmospheric_temperature'

#### <font color='blue'>Adding Global Attributes</font>
* Global attributes are on the dataset.
* We use directly the file identifier to add them.

In [ ]:
current_date = datetime.datetime.today()
ncfid.description = 'Sample netCDF file'
ncfid.institution = 'NASA GSFC'
ncfid.history     = 'File created on ' + current_date.strftime("%B %d, %Y, %H:%M:%S")
ncfid.source      = 'netCDF4 python tutorial'

#### <font color='blue'>Writing Data in the File</font>

In [ ]:
latitudes[:]  =  np.arange(-90,91,2.0)
longitudes[:] =  np.arange(-180,180,2.5)
levels[:]     =  np.arange(0,72,1)

out_frequency = 3   # ouput frequency in hours
num_records   = 5
dates = [beg_date + n*datetime.timedelta(hours=out_frequency) for n in range(num_records)]
times[:] = nc4.date2num(dates, units=times.units, calendar=times.calendar)
for i in range(num_records):
    temp[i,:,:,:] = np.random.uniform(size=(levels.size,
                                            latitudes.size,
                                            longitudes.size))

In [ ]:
print("Time values: \n\t Units: {} \n\t Values: {}".format(times.units, times[:]))

#### <font color='blue'>Printing Dimension Information</font>

List all the dimension information:

In [ ]:
print(ncfid.variables)

In [ ]:
for dim in ncfid.dimensions.values():
     print(dim, dim.isunlimited())

Get the list of dimension name and retrieve info for each dimension:

In [ ]:
for name in ncfid.dimensions.keys():
    dim = ncfid.variables[name]
    print(name, dim.dtype, dim.size)

In [ ]:
for name, dim in ncfid.dimensions.items():
    print(name, dim.size, dim.isunlimited())

#### <font color='blue'>Printing File Attributes</font>

Get the global file attributes

In [ ]:
for att in ncfid.ncattrs():
    print("{:>15}: {}".format(att, ncfid.getncattr(att)))

In [ ]:
for att in ncfid.ncattrs():
    print("{:>15}: {}".format(att, getattr(ncfid, att)))

Global attributes as a dictionary:

In [ ]:
print(ncfid.__dict__)

#### <font color='blue'>Printing Variable Information</font>

List variable information but exclude dimensions:

In [ ]:
for name in ncfid.variables.keys():
    if (name not in ncfid.dimensions.keys()):
       data = ncfid.variables[name]
       print(name, data.units, data.shape, data.dtype, data.dimensions)

In [ ]:
for name, var in ncfid.variables.items():
    if (name not in ncfid.dimensions.keys()):
       print(name, var.units, var.shape, var.dtype, var.dimensions)

You can write a function to print variable attribute:

In [ ]:
def print_ncattr(fid, key):
    """
        Prints the NetCDF file attributes for a given key

        Parameters: 
            * fid:  netCDF file identifier
            * key:  unicode (a valid netCDF4.Dataset.variables key)
    """
    try:
        print('{}  -->'.format(key))
        print("\t {:>15}: {}".format("type", fid.variables[key].dtype))
        for attr in fid.variables[key].ncattrs():
            print('\t {:>15}: {}'.format(attr, fid.variables[key].getncattr(attr)))
    except KeyError:
        print("\t WARNING: {} does not contain variable attributes".format(key))

In [ ]:
print(print_ncattr.__doc__)

In [ ]:
for name in ncfid.variables.keys():
    print_ncattr(ncfid, name)

In [ ]:
print_ncattr(ncfid, "Wind")

In [ ]:
for name, var in ncfid.variables.items():
    print('{}  -->'.format(name))
    print("\t {:>15}: {}".format("type", var.dtype))
    for attr in var.ncattrs():
        print('\t {:>15}: {}'.format(attr, var.getncattr(attr)))

#### <font color='blue'>Create Groups</font>
* We can organize data in hierarchical groups, which are analogous to directories in a filesystem. 
* Groups serve as containers for variables, dimensions and attributes, as well as other groups.
* Use the method `createGroup` to create groups.

In [ ]:
fcstgrp  = ncfid.createGroup('forecasts')
fcstgrpm = ncfid.createGroup('forecasts/model')

List the groups:

In [ ]:
print(ncfid.groups)

In [ ]:
def walk_group_tree(top):
    """
       Python generator that is used to walk the directory tree.
    """
    values = top.groups.values()
    yield values
    for value in top.groups.values():
        for children in walk_group_tree(value):
            yield children

List of the created groups in the dataset:

In [ ]:
for children in walk_group_tree(ncfid):
    for child in children:
        print(child)

##### Add variable to a group

In [ ]:
tempm = ncfid.createVariable('forecasts/model/temp','f4', \
                                  ('time','lev','lat','lon',))

tempm.long_name       = 'temperature (model)'
tempm.units           = 'K'
tempm.valid_min       = 200.
tempm.valid_max       = 350.
tempm.standard_name   = 'atmospheric_temperature'

tempm[0:num_records,:,:,:] = np.random.uniform(size=(num_records,
                                                     levels.size,
                                                     latitudes.size,
                                                     longitudes.size))

ps = fcstgrpm.createVariable('PS', 'f4', ('time', 'lat','lon',))
ps.long_name = 'surface_pressure'
ps.units = 'Pa'
ps.valid_min = -1.0e15
ps.valid_max = 1.0e15
ps[0:num_records,:,:] = 1500.0*np.random.uniform(size=(num_records, latitudes.size,
                                                       longitudes.size))

In [ ]:
print(ncfid["forecasts/model"])

In [ ]:
print(ncfid["forecasts/model/temp"])

#### <font color='blue'>Close the file</font>

In [ ]:
ncfid.close()

## <font color='red'>Data Compression</font>

- netCDF4 provides easy methods to compress data. 
- When you create variables, you can turn on data compression by setting the keyword argument `zlib=True`:

```python
temp = ncfid.createVariable('temp','f4', \
                            ('time','lev','lat','lon',),
                            zlib=True)
```

- The `complevel` keyword argument toggles the compression ratio and speed. 
- Options range from 1 to 9 (1 being the fastest with least compression and 9 being the slowest with most compression. Default is 4). 
- Additionally, you may also specify the precision of your data using the `least_significant_digit` keyword argument.
-  By specifying the least significant digit, you can further enhance the data compression. This just gives netCDF more freedom when it packs the data into your harddrive. 

```python
temp = ncfid.createVariable('temp','f4', \
                            ('time','lev','lat','lon',),
                            zlib=True,
                            least_significant_digit=4)
```

## <font color="red"> Reading netCDF Files</font>

### Reading Individual Files

In [ ]:
with nc4.Dataset(ncFileName, mode='r') as ncfid:
     time   = ncfid.variables['time'][:]
     lev    = ncfid.variables['lev'][:]
     lat    = ncfid.variables['lat'][:]
     lon    = ncfid.variables['lon'][:]
     temp   = ncfid.variables['temp'][:]
     grpid1 = ncfid.groups['forecasts']
     grpid2 = grpid1.groups['model']
     tempm  = grpid2.variables['temp'][:]
    
     # Print variable information
     print_ncattr(ncfid,  'time')
     print_ncattr(ncfid,  'temp')
     print_ncattr(grpid2, 'temp')
    
     dates = nc4.num2date(time, ncfid.variables['time'].units)

     print("-"*50)
     temp_var = ncfid.variables['temp']
     print("Type of temp_var: {}".format(type(temp_var)))
     print("-"*50)
     print(temp_var)
     print("-"*40)

In [ ]:
print("Type of lon:  {}".format(type(lon)))
print("Type of temp: {}".format(type(temp)))

**Important**

While reading data from a file:

- If you do not include `[:]` at the end of `variables[var_name]`, you are getting a variable object.
- If you include `[:]` (or `[:,:]`, `[0, i:j, :]`, etc.) at the end of `variables[var_name]`, you are getting the Numpy array containing the data.

In [ ]:
print("tem_var: ", temp_var)

**Note that the variable object `temp_var` is only accessible as long as the file is opened.**

In [ ]:
print("Time: ", time)

We can get the time values as dates:

In [ ]:
print([date.strftime('%Y-%m-%d %H:%M:%S') for date in dates[:]])

In [ ]:
print("Longitude values: ", lon)

In [ ]:
print("Latitude values: ", lat)

In [ ]:
print("Temperature info: ")
print(temp.shape)
print(np.min(temp), np.max(temp))

In [ ]:
print("Temperature (model) info: ")
print(tempm.shape)
print(np.min(tempm), np.max(tempm))

**To extract data from a scalar variable `v` with no associated dimensions, use `numpy.asarray(v)` or `v[...]`. The result will be a numpy scalar array.**

### Reading Multi-File Dataset

- If you have multiple files that have the same variable and the same unlimited dimension, you can read them in simultaneous using `MFDataset`. 
- Instead of using a single filename to create a `Dataset` instance, create a `MFDataset` instance with either a list of filenames, or a string with a wildcard (which is then converted to a sorted list of files using the python `glob` module).
- `MFDataset` uses file globbing to patch together all the files into one big Dataset.
- It can only aggregate the data along the leftmost dimension of each variable.
- It can be slow!
- This is only available for `NETCDF3_64BIT`, `NETCDF3_CLASSIC` or `NETCDF4_CLASSIC` formats.

Assume that you create the files:

In [ ]:
for nf in range(10):
    with nc4.Dataset("mftest%s.nc" % nf, "w", format="NETCDF4_CLASSIC") as f:
         _ = f.createDimension("x", None)
         x = f.createVariable("x", "i", ("x",))
         x[0:10] = numpy.arange(nf*10,10*(nf+1))

You can read all the files and access the variable `x`:

In [ ]:
f = nc4.MFDataset("mftest*nc")
all_x = f.variables["x"][:]
print(all_x)

- The command `all_x = f.variables["x"][:]` loads all into memory all the values of the variable `x` from all the netCDF files.
- To potentially avoid crashing your computer, it is better to load small parts at a time by replacing `[:]` with slices, for example, `[0:2]`, `[2:4]`, etc.

### <font color='red'>Useful Methods</font>

#### Methods acting on the file

| Method Name | Description |
| --- | --- |
| `ncattrs()` | List all the global attributes |
| `getncattr(attr)` | Get the value of the attribute `attr` |



#### Methods acting on a variable

| Method Name | Description |
| --- | --- |
| `ncattrs()` | List all the attributes of a variable|
| `getncattr(attr)` | Get the value of the attribute `attr` |
| `setncatts(attr, val)` | Set the value `val` to the attribute `attr`|
| `renameDimension(oldname, newname)` | Rename a dimension named `oldname` to `newname` |
| `renameVariable(oldname, newname)` | Rename a variable named `oldname` to `newname` |

**Print the dimensions:**

In [ ]:
with nc4.Dataset(ncFileName, mode='r') as ncfid:
     for name, dim in ncfid.dimensions.items():
         print("{}: {}".format(name, len(dim)))

**Print the global attributes:**

In [ ]:
with nc4.Dataset(ncFileName, mode='r') as ncfid:
     for attr in ncfid.ncattrs():
         print("{}: {}".format(attr, ncfid.getncattr(attr)))

**Print all variables and their attributes:**

In [ ]:
with nc4.Dataset(ncFileName, mode='r') as ncfid:
     for name, var in ncfid.variables.items():
         print("{}:".format(name))
         print("{:>20}: {}".format("Dimension", var.dimensions))
         print("{:>20}: {}".format("Dimension value", var.shape))
         print("{:>20}: {}".format("Type", var.dtype))
         for attr in var.ncattrs():
             print("{:>20}: {}".format(attr, var.getncattr(attr)))

**Mimic `ncdump -h` Command**

In [ ]:
def my_ncdump(file_name):
    with nc4.Dataset(file_name, mode='r') as ncfid:
         for name, var in ncfid.variables.items():
             print("{}:".format(name))
             print("{:>20}: {}".format("Dimension", var.dimensions))
             print("{:>20}: {}".format("Dimension value", var.shape))
             print("{:>20}: {}".format("Type", var.dtype))
             for attr in var.ncattrs():
                 print("{:>20}: {}".format(attr, var.getncattr(attr)))
         print()
         print("Gobal attributes")
         for attr in ncfid.ncattrs():
             print("{:>20}: {}".format(attr, ncfid.getncattr(attr)))

In [ ]:
my_ncdump(ncFileName)

### <font color='red'>Updating a Variable in an Existing netCDF File</font>

In [ ]:
with nc4.Dataset(ncFileName, mode='a') as ncfid:
     temp = ncfid.variables['temp'][:]
     data = temp[:]
     data = 1.1*data + 100.0
     temp[:] = data

In [ ]:
print(temp.shape)
print(np.min(temp), np.max(temp))

### <font color='red'>Copy the Content of a netCDF File</font>

In [ ]:
def copy_nc_file(src_file, dst_file):
    with nc4.Dataset(src_file) as srcid:
         with Dataset(dst_file, mode='w') as dstid:
              # Create the dimensions of the file
              for name, dim in srcid.dimensions.items():
                  dstid.createDimension(name, len(dim) if not dim.isunlimited() else None)

              # Copy the global attributes
              dstid.setncatts({a:srcid.getncattr(a) for a in srcid.ncattrs()})

              # Create the variables in the file
              for name, var in srcid.variables.items():
                  dstid.createVariable(name, var.dtype, var.dimensions)

                  # Copy the variable attributes
                  dstid.variables[name].setncatts({a:var.getncattr(a) for a in var.ncattrs()})

                  # Copy the variables values
                  dstid.variables[name][:] = srcid.variables[name][:]

### <font color='red'>Compare two netCDF Files</font>

In [ ]:
def rms(V):
    return np.sqrt(np.mean(np.square(V)))

def compare_two_nc_files(fileA, fileB):
    machine_eps = 1.0e-25

    print("-" *100)
    print("Comparing the two nc files:")
    print("  * ", fileA)
    print("  * ", fileB)
    print("-" *100)

    with nc4.Dataset(fileA, mode='r', format='NETCDF4') as fidA:
         with nc4.Dataset(fileB, mode='r', format='NETCDF4') as fidB:
              listVars = fidA.variables.keys()
              dimVars  = fidA.dimensions.keys()

              print("            Machine precision: ", machine_eps)
              same = True
              for varName in listVars:
                  if varName not in dimVars:
                     valsA = fidA.variables[varName][:]
                     valsB = fidB.variables[varName][:]
                     diff = np.sum(valsA - valsB)
                     print("Variable Name and Sum of Difference: {:<15} {:30.27f}".format(varName, diff))
                     err = rms(valsA - valsB)
                     print(err)
                     if (err > machine_eps):
                        print("     The variable {} differs in the two files".format(varName))
                        same = False
                        break


    print("-" *100)
    if same:
       print(" -----> The two files are identical!")
    else:
       print(" -----> The two files are different!")

    print("-" *100)


In [ ]:
compare_two_nc_files(ncFileName, ncFileName)

## <font color="red">Application 1: </font> Create a climatology file

- We have a file that has several variables but only one record (one monthly value).
- We want to create a new file that is the same as the first one but contains 12 identical records (one per month).

## <font color="red">Application 2: </font> Global Average Time Series

- We have a collection of files covering multiple years.
- We consider one variable and compute its yearly mean.
- We plot the yearly mean values as function of years.

## <font color="red">Application 3: </font> Create Monthly Mean Climatologies

- We have a collection of monthly files covering multiple years.
- We want to create monthly climatology files.

## <font color="red">Example</font>

We want to use the netCDF file:

https://www.unidata.ucar.edu/software/netcdf/examples/sresa1b_ncar_ccsm3-example.nc

to plot the surface air temperature (variable `tas`) and the zonal mean height of the wind (variable `ua`).


The metadata of the file is located at:

https://www.unidata.ucar.edu/software/netcdf/examples/sresa1b_ncar_ccsm3-example.cdl

In [ ]:
# Get the remote  file
nc_file = "sresa1b_ncar_ccsm3-example.nc"
url = "https://www.unidata.ucar.edu/software/netcdf/examples/"
import urllib.request
urllib.request.urlretrieve(url+nc_file, nc_file)

In [ ]:
# Open the netCDF file and read surface air temperature
with nc4.Dataset(nc_file,'r') as ncid:
     lons      = ncid.variables['lon'][:] # longitude grid points
     lats      = ncid.variables['lat'][:] # latitude grid points
     levs      = ncid.variables['plev'][:] # pressure leves
     surf_temp = ncid.variables['tas'][:]
     uwind     = ncid.variables['ua'][:]

In [ ]:
print("Shape of lons:      ", np.shape(lons), lons[0], lons[-1])
print("Shape of lats:      ", np.shape(lats), lats[0], lats[-1])
print("Shape of levs:      ", np.shape(levs), levs[0], levs[-1])
print("Shape of surf_temp: ", np.shape(surf_temp))
print("Shape of uwind:     ", np.shape(uwind))

**Load Plotting Modules**

**<font color='blue'> Uncomment the cell below if you are on Google Colab</font>**

In [ ]:
#!apt-get install libproj-dev proj-data proj-bin
#!apt-get install libgeos-dev
#!pip install cython
#!pip install cartopy

---

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

**Plot of Surface Temperature**

In [ ]:
fig = plt.figure(figsize=(9, 5))
map_projection = ccrs.PlateCarree()
ax = fig.add_subplot(1, 1, 1, projection=map_projection)

im = ax.contourf(lons, lats, surf_temp[0,:,:], transform=map_projection)
ax.coastlines()

ax.set_xticks(np.linspace(-180, 180, 5), crs=map_projection)
ax.set_yticks(np.linspace(-90, 90, 5), crs=map_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)

cb = fig.colorbar(im, orientation='vertical', shrink=0.75)

ax.set_global()
plt.show()

**Plot Zonal Mean Height of Wind**

In [ ]:
# Compute the zonal mean height
zonal_mean_height = np.mean(uwind[0,:,:,:], axis=2)

# Select contour levels
ncountours = 10
fac = 0.005
min_val = (1.0-fac)*np.min(zonal_mean_height)
max_val = (1.0+fac)*np.max(zonal_mean_height)
clevs = np.linspace(min_val, max_val, ncountours)

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1)

# map contour values to colors
norm=matplotlib.colors.BoundaryNorm(clevs, ncolors=256, clip=False)
    
# draw the contours with contour labels
CS = ax.contour(lats, levs, zonal_mean_height, levels=clevs)
ax.clabel(CS,inline=1, fontsize=10, colors='black')

# draw the (filled) contours
contour = ax.contourf(lats, levs, zonal_mean_height, levels=clevs, norm=norm)

# Draw colorbar
fmt = matplotlib.ticker.FormatStrFormatter("%3.2g")
cbar = fig.colorbar(contour, ax=ax, orientation='horizontal', shrink=0.8,
                    ticks=clevs, format=fmt)
cbar.set_label('m s-1')

ax.set_yscale('log')
ax.set_xlabel("Latitude (degrees)")
ax.set_ylabel("Pressure (Pa)")
plt.show()